In [1]:
import sympy as sp
import numpy as np
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi, lambdify
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify
from scipy.integrate import solve_ivp

In [2]:
from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

# Simulating Multi-link System

## Solving the Dynamics

In [3]:
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 m_1,
 m_2,
 m_3,
 g) =         symbols("""t, 
                         theta_1 
                         theta_2 
                         theta_3 
                         l_1 
                         l_2 
                         l_3
                         m_1
                         m_2
                         m_3
                         g""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [4]:
def T(theta, x, y):
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

In [5]:
# Transformation Matricies
T_01 = T(theta_1, 0,   0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)

In [6]:
# Zero Positions
M1 = Matrix([[l_1], [0], [1]])
M2 = Matrix([[l_2], [0], [1]])
M3 = Matrix([[l_3], [0], [1]])

In [7]:
# Forward Kinematics
FK1 = (T_01 * M1)[:-1,0]
FK2 = simplify((T_01 * T_12 * M2)[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * M3)[:-1,0])

In [8]:
# Jacobian 
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
J = Matrix([J1,J2,J3])

In [9]:
# Velocities
v1 = diff(FK1,t)
v2 = diff(FK2,t)
v3 = diff(FK3,t)

In [10]:
# Calculate mass matrix from Jacobian
mass_newton = J.transpose() @ sp.diag(m_1,m_1,m_2,m_2,m_3,m_3) @ J
mass_newton = simplify(mass_newton)

In [11]:
# Kinetic Energy
KE1 = (1/2)*m_1*(v1.T * v1)[0]
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE3 = (1/2)*m_3*(v3.T * v3)[0]

In [12]:
# Potential Energy
P1 = m_1*g*FK1[1]
P2 = m_2*g*FK2[1]
P3 = m_3*g*FK3[1]

In [13]:
# Lagrangian 
L = KE1 + KE2 + KE3 - P1 - P2 - P3
L = simplify(L)


In [14]:
# Equations of Motion
EOM_theta_1 = diff(diff(L, Derivative(theta_1, t)), t) - diff(L, theta_1)
EOM_theta_1 = nsimplify(EOM_theta_1)
EOM_theta_2 = diff(diff(L, Derivative(theta_2, t)), t) - diff(L, theta_2)
EOM_theta_2 = nsimplify(EOM_theta_2)
EOM_theta_3 = diff(diff(L, Derivative(theta_3, t)), t) - diff(L, theta_3)
EOM_theta_3 = nsimplify(EOM_theta_3)

In [15]:
# Extract mass matrix from legrangian
mass = sp.symarray('',(3,3))

mass[0,0] = EOM_theta_1.expand().coeff(Derivative(theta_1,t,t))
mass[0,1] = EOM_theta_1.expand().coeff(Derivative(theta_2,t,t))
mass[0,2] = EOM_theta_1.expand().coeff(Derivative(theta_3,t,t))

mass[1,0] = EOM_theta_2.expand().coeff(Derivative(theta_1,t,t))
mass[1,1] = EOM_theta_2.expand().coeff(Derivative(theta_2,t,t))
mass[1,2] = EOM_theta_2.expand().coeff(Derivative(theta_3,t,t))

mass[2,0] = EOM_theta_3.expand().coeff(Derivative(theta_1,t,t))
mass[2,1] = EOM_theta_3.expand().coeff(Derivative(theta_2,t,t))
mass[2,2] = EOM_theta_3.expand().coeff(Derivative(theta_3,t,t))

mass_lagrange = Matrix(mass)
mass_lagrange = simplify(mass_lagrange)

In [16]:
# Extract other terms
accel_state_vector = Matrix([[Derivative(theta_1, t, t)], [Derivative(theta_2, t, t)], [Derivative(theta_3, t, t)]])

EOM1_leftover = simplify(EOM_theta_1.expand() - (mass_lagrange[0,:]*accel_state_vector)[0])
EOM2_leftover = simplify(EOM_theta_2.expand() - (mass_lagrange[1,:]*accel_state_vector)[0])
EOM3_leftover = simplify(EOM_theta_3.expand() - (mass_lagrange[2,:]*accel_state_vector)[0])

In [17]:
# Extract gravitational matrix
G_1 = EOM1_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_2 = EOM2_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_3 = EOM3_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])

In [18]:
# Extract coriolis matrix  
C_1 = EOM1_leftover - G_1
C_2 = simplify(EOM2_leftover - G_2)
C_3 = simplify(EOM3_leftover - G_3)

In [19]:
# Is newton the same as Lagrange? 
sp.Eq(mass_newton,mass_lagrange)

True

## Simulating

In [20]:
# Create numerical functions
M =   lambdify((theta_1, theta_2, theta_3, Derivative(theta_1, t), Derivative(theta_2, t),Derivative(theta_3, t), l_1, l_2, l_3, m_1, m_2, m_3), mass_lagrange)
EOM = lambdify((theta_1, theta_2, theta_3, Derivative(theta_1, t), Derivative(theta_2, t),Derivative(theta_3, t), l_1, l_2, l_3, m_1, m_2, m_3), 
               Matrix([[EOM1_leftover.subs([(g, 9.8)])], [EOM2_leftover.subs([(g, 9.8)])],[EOM3_leftover.subs([(g, 9.8)])]]))

nFK1 = lambdify((theta_1, theta_2, theta_3,l_1, l_2, l_3, m_1, m_2, m_3),FK1)
nFK2 = lambdify((theta_1, theta_2, theta_3,l_1, l_2, l_3, m_1, m_2, m_3),FK2)
nFK3 = lambdify((theta_1, theta_2, theta_3,l_1, l_2, l_3, m_1, m_2, m_3),FK3)

In [21]:
# Parameters
m = [1,1,2]
m1,m2,m3 = m
l = [1,1,1.5]
l1,l2,l3 = l

In [22]:
# Dynamics Function
def dynamics(t, state, l, m):
    theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3 = state
    m_1,m_2,m_3 = m
    l_1,l_2,l_3 = l
    dydt = np.linalg.inv(M(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))@(-EOM(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))
    return [dtheta_1, dtheta_2, dtheta_3, dydt[0][0], dydt[1][0] , dydt[2][0]]

In [23]:
state0 = [np.pi/2,0,0,0,0,0]
dynamics(0, state0, l, m)

[0,
 0,
 0,
 -6.000769315822004e-16,
 6.000769315821969e-16,
 1.5777218104420236e-30]

In [24]:
t_end = 10
dt = 0.0001
time = np.linspace(0,t_end,int(t_end/dt))

sol = solve_ivp(lambda t, y: dynamics(t, y, l, m),
                [0,t_end],state0, 
                t_eval = time, 
                rtol=1e-8, atol = 1e-8)

In [25]:

fig = plt.figure();
ax = plt.gca();
ax.plot(sol.t, sol.y[0,:],label=r"$\theta_1$")
ax.plot(sol.t, sol.y[1,:],label=r"$\theta_2$")
ax.plot(sol.t, sol.y[2,:],label=r"$\theta_3$")
ax.legend()
plt.show()
# plt.ylabel('$x$')
# plt.xlabel('time')

In [26]:
stop = False

def on_close(event):
    global stop 
    stop = True

fig = plt.figure()
fig.canvas.mpl_connect('close_event', on_close)

skip = 100
p0 = np.zeros(len(time[0:-1:skip]))
st = time[0:-1:skip]
theta1 = sol.y[0,0:-1:skip]
theta2 = sol.y[1,0:-1:skip]
theta3 = sol.y[1,0:-1:skip]




for i in range(len(p0)):
    plt.clf();
    ax = plt.gca();

    ax.plot([p0[i],nFK1(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0],nFK2(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0],nFK3(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0]],
            [p0[i],nFK1(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0],nFK2(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0],nFK3(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0]])
    ax.scatter([p0[i],nFK1(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0],nFK2(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0],nFK3(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[0][0]],
               [p0[i],nFK1(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0],nFK2(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0],nFK3(theta1[i],theta2[i],theta3[i],l1,l2,l3,m1,m2,m3)[1][0]])
    ax.set_xlim([-5,5])
    ax.set_ylim([-5,5])
    ax.text(3,4,str(st[i]))
    
    plt.pause(0.01)
    plt.show()
    if(stop):
        break

# Rimless Wheel

In [3]:
(k,
 alpha,
 theta0,
 theta,
 dtheta,
 l,  
 g,
 t,
 m) = symbols(r'k, alpha, theta_0, theta, \dot{\theta}, l, g,t, m')
# I = m*l**2
# x = Matrix([[dtheta],[(g/l)* sin(theta) - k*(theta-theta0)/I]])
# x

In [133]:
def rimless_wheel(t,x,args):
    theta,dtheta = x
    m, g, l, theta0, k = args
    return [dtheta, -(g/l)* sin(theta) - k*(theta-theta0)/m/l**2]




In [201]:
alpha = np.pi/6
arg = [1, 9.8, 1, alpha, 1]
time = np.linspace(0,1,100)


def step_event(t,state): 
    theta, dtheta = state
    return theta - alpha
    
    
def collision_reset(state,step):
    theta, omega = state
    if(step != 0):
        theta = - alpha
        omega = omega*np.cos(2*alpha)
    return [theta, omega]

step_event.terminal = True

state_sol = [-alpha-0.0001,0]
time_sol = 0
steps = 5

theta = np.array(state_sol[0])
dtheta = np.array(state_sol[1])
time = np.array([0])
N = 1000
t_end = 10

steptime = []
vstep = []

for i in range(steps):
    sol = solve_ivp(lambda t,x: rimless_wheel(t,x,arg),
                    [time_sol,time_sol + t_end],
                    collision_reset(state_sol,i), 
                    t_eval = np.linspace(time_sol,time_sol + t_end,N),
                    events = step_event)
    time_sol = sol.t[-1]
    state_sol = sol.y[:,-1]
    theta = np.hstack([theta,sol.y[0,:-1]])
    dtheta = np.hstack([dtheta, sol.y[1,:-1]])
    time = np.hstack([time,sol.t[:-1]])
    steptime.append(sol.t_events[0][0])
    vstep.append(sol.y_events[0][0])




In [202]:
returnmap = np.zeros([2,len(vstep)-1])

for i in range(len(vstep)-1):
    returnmap[:,i] = np.array([vstep[i][1], vstep[i+1][1]])


In [203]:
returnmap

array([[0.88523966, 0.99179489, 1.0137992 , 1.01805216],
       [0.99179489, 1.0137992 , 1.01805216, 1.01671843]])

In [206]:
plt.close();
fig2 = plt.figure()
ax = plt.gca()
ax.scatter(returnmap[0],returnmap[1],20);
ax.plot(returnmap[0],returnmap[1],linewidth=1);
ax.plot(np.linspace(0,1),np.linspace(0,1),'--r',linewidth=1);


In [208]:
plt.close();
fig3 = plt.figure()
ax = plt.gca();
ax.plot(theta,dtheta)

In [209]:
plt.close();
fig1 = plt.figure()
ax = plt.gca();
ax.plot(time,theta)
ax.plot(time,dtheta)
ax.plot([0,time[-1]],[alpha,alpha])


In [210]:
plt.close("all");

stop = False

def on_close(event):
    global stop 
    stop = True


fig2 = plt.figure();
fig2.canvas.mpl_connect('close_event', on_close)

step = 0;
for i in range(len(theta)):
    if(time[i] < time [-2]):
        if(time[i+1] >= steptime[step]):
            step = step+1
        
    plt.clf()
    ax = plt.gca()
    px = np.sin(theta[i])
    py = np.cos(theta[i])
    dx = 2 * np.sin(alpha) * step
    ax.plot([dx, px+dx],[0,py])
    ax.scatter([dx, px+dx],[0,py])
    ax.plot([-1.5+dx+px,1.5+dx+px],[0,0],'k')
    plt.grid()
    ax.set_xlim([-1.5+dx+px,1.5+dx+px])
    ax.set_ylim([-1.5,1.5])
    str_time = "{:.5f}".format(time[i])
    plt.title(str_time)
    plt.draw()
    plt.show()
    plt.pause(0.01)
    if(stop):
        stop = False
        break


In [19]:

alpha = np.pi/6
theta_0 = np.linspace(-100,-alpha,100);
k = np.linspace(0,100,100);
K, THETA_0 = np.meshgrid(k,theta_0)

In [20]:
m = 1
l = 5
theta_star = 2*np.sqrt(-K*alpha*THETA_0/m/l**2)*1/np.tan(2*alpha)

In [39]:



plt.imshow(theta_star)
# plt.matshow(theta_star)
#plt.xticks(np.linspace(10,100,10),np.linspace(10,100,10))


In [ ]:
k